# Convolutional Neural Network (CNN) Implementation

This notebook demonstrates a complete implementation of a Convolutional Neural Network for image classification.

## Dataset
We'll use the MNIST dataset (handwritten digits) for demonstration.

## Topics Covered:
1. Image Data Preprocessing
2. CNN Architecture Design
3. Convolutional Layers
4. Pooling Layers
5. Training and Evaluation
6. Visualization of Filters

## 1. Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

print(f"TensorFlow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")

## 2. Load and Explore Data

In [ ]:
# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(f"Training set shape: {X_train.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Test set shape: {X_test.shape}")
print(f"Test labels shape: {y_test.shape}")
print(f"\nNumber of classes: {len(np.unique(y_train))}")
print(f"Classes: {np.unique(y_train)}")

## 3. Visualize Sample Images

In [ ]:
# Display sample images
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
axes = axes.ravel()

for i in range(10):
    axes[i].imshow(X_train[i], cmap='gray')
    axes[i].set_title(f"Label: {y_train[i]}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

# Class distribution
plt.figure(figsize=(10, 5))
unique, counts = np.unique(y_train, return_counts=True)
plt.bar(unique, counts)
plt.xlabel('Digit')
plt.ylabel('Frequency')
plt.title('Class Distribution in Training Set')
plt.xticks(unique)
plt.grid(axis='y', alpha=0.3)
plt.show()

## 4. Data Preprocessing

In [ ]:
# Reshape data to include channel dimension
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

# Normalize pixel values to [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# One-hot encode labels
y_train_encoded = to_categorical(y_train, num_classes=10)
y_test_encoded = to_categorical(y_test, num_classes=10)

print(f"Training set shape after preprocessing: {X_train.shape}")
print(f"Training labels shape after encoding: {y_train_encoded.shape}")
print(f"\nPixel value range: [{X_train.min()}, {X_train.max()}]")

## 5. Build CNN Architecture

In [ ]:
# Initialize the CNN
model = Sequential()

# First Convolutional Block
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Second Convolutional Block
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flatten layer
model.add(Flatten())

# Fully Connected Layers
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Output layer
model.add(Dense(10, activation='softmax'))

# Display model architecture
model.summary()

## 6. Compile the Model

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("Model Compiled Successfully")

## 7. Data Augmentation

In [ ]:
# Create data augmentation generator
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

datagen.fit(X_train)

# Visualize augmented images
sample_image = X_train[0].reshape(1, 28, 28, 1)
fig, axes = plt.subplots(1, 5, figsize=(15, 3))

i = 0
for batch in datagen.flow(sample_image, batch_size=1):
    axes[i].imshow(batch[0].reshape(28, 28), cmap='gray')
    axes[i].set_title(f'Augmented {i+1}')
    axes[i].axis('off')
    i += 1
    if i >= 5:
        break

plt.tight_layout()
plt.show()

## 8. Train the Model

In [ ]:
# Define callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the model
history = model.fit(datagen.flow(X_train, y_train_encoded, batch_size=64),
                    validation_data=(X_test, y_test_encoded),
                    epochs=30,
                    callbacks=[early_stop, reduce_lr],
                    verbose=1)

print("\nTraining Complete!")

## 9. Visualize Training History

In [ ]:
# Plot training & validation accuracy
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.grid(True)

# Plot training & validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## 10. Evaluate the Model

In [ ]:
# Make predictions
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=range(10), yticklabels=range(10))
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

## 11. Visualize Predictions

In [ ]:
# Display correct and incorrect predictions
correct_indices = np.where(y_pred == y_test)[0]
incorrect_indices = np.where(y_pred != y_test)[0]

# Show some correct predictions
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Correct Predictions', fontsize=16)
axes = axes.ravel()

for i in range(10):
    idx = correct_indices[i]
    axes[i].imshow(X_test[idx].reshape(28, 28), cmap='gray')
    axes[i].set_title(f"Pred: {y_pred[idx]}, True: {y_test[idx]}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

# Show some incorrect predictions
if len(incorrect_indices) > 0:
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    fig.suptitle('Incorrect Predictions', fontsize=16)
    axes = axes.ravel()
    
    for i in range(min(10, len(incorrect_indices))):
        idx = incorrect_indices[i]
        axes[i].imshow(X_test[idx].reshape(28, 28), cmap='gray')
        axes[i].set_title(f"Pred: {y_pred[idx]}, True: {y_test[idx]}", color='red')
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

## 12. Visualize Convolutional Filters

In [ ]:
# Get the weights of the first convolutional layer
filters, biases = model.layers[0].get_weights()

# Normalize filter values to 0-1 for visualization
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)

# Plot first 32 filters
n_filters = 32
fig, axes = plt.subplots(4, 8, figsize=(15, 8))
fig.suptitle('First Layer Convolutional Filters', fontsize=16)
axes = axes.ravel()

for i in range(n_filters):
    axes[i].imshow(filters[:, :, 0, i], cmap='gray')
    axes[i].set_title(f'Filter {i+1}')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

## 13. Visualize Feature Maps

In [ ]:
# Create a model to get intermediate layer outputs
layer_outputs = [layer.output for layer in model.layers[:6]]  # First 6 layers
activation_model = keras.models.Model(inputs=model.input, outputs=layer_outputs)

# Get activations for a sample image
sample_image = X_test[0].reshape(1, 28, 28, 1)
activations = activation_model.predict(sample_image)

# Visualize first layer activations
first_layer_activation = activations[0]
fig, axes = plt.subplots(4, 8, figsize=(15, 8))
fig.suptitle('First Layer Feature Maps', fontsize=16)
axes = axes.ravel()

for i in range(32):
    axes[i].imshow(first_layer_activation[0, :, :, i], cmap='viridis')
    axes[i].set_title(f'Map {i+1}')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

## 14. Save the Model

In [ ]:
# Save the model
model.save('cnn_mnist_model.h5')
print("Model saved as 'cnn_mnist_model.h5'")

# To load the model later:
# from tensorflow.keras.models import load_model
# loaded_model = load_model('cnn_mnist_model.h5')

## Summary

### Key Takeaways:
1. **CNN Architecture**: Convolutional layers extract spatial features
2. **Pooling**: Reduces dimensionality while retaining important features
3. **Data Augmentation**: Improves generalization
4. **Batch Normalization**: Stabilizes training
5. **Dropout**: Prevents overfitting

### When to Use CNN:
- Image classification
- Object detection
- Image segmentation
- Face recognition
- Medical image analysis

### Advantages:
- Automatically learns spatial hierarchies
- Parameter sharing reduces model size
- Translation invariant
- Excellent for image data

### CNN vs ANN:
- **CNN**: Best for image/spatial data
- **ANN**: Best for tabular/structured data
- CNN has fewer parameters due to weight sharing
- CNN preserves spatial relationships